In [3]:
#Import of Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import scipy.sparse.linalg

from PIL import Image as Image, ImageDraw as Imk
from IPython.display import Image, display

import scipy 

import dlib as dlib
from sklearn.datasets import load_svmlight_file
import skimage as sk
import skimage.io as io
import face_recognition
import cv2
from google_images_download import google_images_download 

import math
from sklearn import neighbors
import os
import os.path
import glob
import pickle
from face_recognition.face_recognition_cli import image_files_in_folder

### Initializing helper functions to read videos and images

In [59]:
def readVideo(filename, model):
    # Open the input movie file
    input_movie = cv2.VideoCapture(filename)
    length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))
    y = int(input_movie.get(3))
    x = int(input_movie.get(4))
    
    # Create an output movie file (make sure resolution/frame rate matches input video!)
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    cleanFilename = filename[:-4]
    #cleanFilename = "Videos_Output/" + model + "_" + filename + "_output.avi"
    cleanFilename = model + "_" + filename.split('/')[1] + "_output.avi"
    print("Output filename:", cleanFilename)

    output_movie = cv2.VideoWriter(cleanFilename, fourcc = fourcc, fps=24.0, frameSize=(y, x))
    # Load some sample pictures and learn how to recognize them.
    #Deep Learning Library face_encoding called
    
    return length, input_movie, output_movie

In [5]:
def readImageFiles(img_dir):
    #Extract encoding from each image
    #This is a test, once it works, we can apply it to frames extracted from video 
    data_path = os.path.join(img_dir,'*.jpg')
    files = glob.glob(data_path)
    loaded_image = []
    encodings = []
    labels = []
    i = 0
    for filepath in files:
        img = face_recognition.load_image_file(filepath)
        ims = img[:, :, ::-1]
        loaded_image = np.asarray(ims)
        enco = face_recognition.face_encodings(loaded_image)

        ## Checking if there is only one encoding, if there is more/less, then the image will not be included in the testing
        if len(enco) == 1:
            ## todo: either keep this indexing method or keep separate running tally (so as not to have missing vals)
            labels.append(filepath)
            encodings.append(enco)

    encodings = np.asarray(encodings)
    labels = np.asarray(labels)
    return encodings, labels

### Finding accuracy rate

In [6]:
video_train_encodings, video_train_labels = readImageFiles("Train_ImagefromVideo")
google_train_encodings, google_train_labels = readImageFiles("Train_Images")
test_encodings, test_labels = readImageFiles("Test_Images")
unknown_test_encodings, unknown_test_labels = readImageFiles("Test_Images_UnknownFromVideo")

# reshaping
video_train_encodings.resize(len(video_train_encodings), 128)
google_train_encodings.resize(len(google_train_encodings), 128)
test_encodings.resize(len(test_encodings), 128)
unknown_test_encodings.resize(len(unknown_test_encodings), 128)

In [7]:
def directAccuracy(train_encoding, test_encoding, train_label, test_label):
    comparison = []
    for enc in test_encoding:
        match = face_recognition.compare_faces(train_encoding, enc, tolerance=0.50)
        # could make this logic a lot prettier
        name = None
        for idx in range(len(match)):
            if match[idx]:
                name = train_label[idx].split('/')[1].split('_')[0]
                
        comparison.append(name)

    counter = 0
    for i in range(len(test_label)):
        if (comparison[i] == test_label[i].split('/')[1].split('_')[0]): 
            counter += 1
    return comparison, counter/len(test_label)

_, videoAccuracy = directAccuracy(video_train_encodings,test_encodings,\
                              video_train_labels, test_labels)
_, googleAccuracy = directAccuracy(google_train_encodings,test_encodings,\
                               google_train_labels, test_labels)

videoFalsePos, _ = directAccuracy(video_train_encodings,\
                               unknown_test_encodings,\
                              video_train_labels,\
                              unknown_test_labels)
googleFalsePos, _ = directAccuracy(google_train_encodings,\
                                unknown_test_encodings,\
                               google_train_labels,\
                               unknown_test_labels)

print("The video training set has an accuracy rate of", videoAccuracy)
print("The Google training set has an accuracy rate of", googleAccuracy)

def falsePosRate(falsePosList):
    counter = 0
    for match in falsePosList:
        if match != None:
            counter += 1
    return counter/len(falsePosList)

print("The video training set has a false positive rate of", falsePosRate(videoFalsePos))
print("The Google training set has a false positive rate of", falsePosRate(googleFalsePos))


The video training set has an accuracy rate of 0.6176470588235294
The Google training set has an accuracy rate of 0.3235294117647059
The video training set has a false positive rate of 0.19047619047619047
The Google training set has a false positive rate of 0.0


### Updating video with classification outputs

In [55]:
def directToVideo(filename):
    # TODO: automate
    length, input_movie, output_movie = readVideo(filename, "direct")
    known_faces, labels = readImageFiles("Train_ImagefromVideo")
    known_faces.resize(known_faces.shape[0], known_faces.shape[2])

    # Initialize some variables
    face_locations = []
    face_encodings = []
    face_names = []
    frame_number = 0

    #while input_movie.isOpened():
    while True: 
        # Grab a single frame of video
        ret, frame = input_movie.read()
        #print(frame.shape)
        frame_number += 1

        # Quit when the input video file ends
        if not ret:
            print("break")
            break

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_frame = frame[:, :, ::-1]

        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
        face_names = []
        for face_encoding in face_encodings:
            print("Found a face")
            # See if the face is a match for the known face(s)
            match = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.50)
            # could make this logic a lot prettier
            name = None
            for match_index, match_boolean in enumerate(match):
                if match_boolean:
                    name = labels[match_index].split('/')[1].split('_')[0]
                    break

            face_names.append(name)

        # Label the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            if not name:
                continue

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

        # Write the resulting image to the output video file
        print("Writing frame {} / {}".format(frame_number, length))

        # confirmed: frames are being read fine, the issue is writing each frame
        output_movie.write(frame)

    print("All done")
    # All done!
    input_movie.release()
    output_movie.release()
    cv2.destroyAllWindows()
    

## K-Nearest Neighbors Method

### Helper functions to train and predict

In [9]:
# Trains a k-nearest neighbors classifier for face recognition.
def train(image_filenames, n_neighbors=None, knn_algo='ball_tree', verbose=False):
    X = []
    y = []

    # Loop through each person in the training set

    
    # Loop through each training image for the current person
    for img_path in image_filenames:
        image = face_recognition.load_image_file(img_path)
        face_bounding_boxes = face_recognition.face_locations(image)

        if len(face_bounding_boxes) != 1:
            # If there are no people (or too many people) in a training image, skip the image.
            if verbose:
                print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
        else:
            # Add face encoding for current image to the training set
            X.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
            y.append(img_path) # TODO: rename images so that this looks better, no need for class_dir

    # Determine how many neighbors to use for weighting in the KNN classifier
    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(X))))
        if verbose:
            print("Chose n_neighbors automatically:", n_neighbors)

    # Create and train the KNN classifier
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)

    # Return the trained KNN classifier
    return knn_clf

# Recognizes faces in given image using a trained KNN classifier
def predict(X_img, knn_clf=None, distance_threshold=0.6):
    # Load a trained KNN model (if one was passed in)
    if knn_clf is None:
        raise Exception("Must supply knn classifier")

    # Load image file and find face locations
    #X_img = face_recognition.load_image_file(X_img_path)
    X_face_locations = face_recognition.face_locations(X_img)

    # If no faces are found in the image, return an empty result.
    if len(X_face_locations) == 0:
        return []

    # Find encodings for faces in the test iamge
    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)

    # Use the KNN model to find the best matches for the test face
    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
    are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(X_face_locations))]

    # Predict classes and remove classifications that aren't within the threshold
    # Output in format [(name, (location)) for all faces in photo]
    return [(pred, loc) if rec else ("unknown", loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches)]


### Finding accuracy rate

In [50]:
# Initialization

output = {}
train_image_filenames = glob.glob('Train_ImagefromVideo/*.jpg')
knn_clf = train(train_image_filenames)
test_image_filenames = glob.glob('Test_Images/*.jpg')
for idx, img in enumerate(test_image_filenames):
    X_img = face_recognition.load_image_file(img)
    X_enc = face_recognition.face_encodings(X_img)
    if len(X_enc) == 1:
        predictions = predict(X_img,knn_clf = knn_clf)
        output[idx] = predictions

In [11]:
# Computation

counter = 0
for key in output:
    #print (output[key][0][0].split('/')[1].split('_')[0])
    if output[key][0][0] != 'unknown':
        outputKey = output[key][0][0].split('/')[1].split('_')[0]
        if outputKey == test_image_filenames[key].split('/')[1].split('_')[0]:
            counter += 1
print("K-nn Accuracy rate:", counter/len(output))

K-nn Accuracy rate: 0.8058252427184466


### Testing for false positives

In [12]:
# Training from video images
output_unknown = {}
unknown_test_image_filenames = glob.glob('Test_Images_Unknown/*.jpg')

for idx, img in enumerate(unknown_test_image_filenames):
    X_img_unknown = face_recognition.load_image_file(img)
    X_enc_unknown = face_recognition.face_encodings(X_img)
    if len(X_enc_unknown) == 1:
        predictions_unknown = predict(X_img,knn_clf = knn_clf)
        output_unknown[idx] = predictions

print("The video training set has a false positive rate of", len(output_unknown)/len(unknown_test_image_filenames))


# Training from Google images
output_unknown = {}
train_image_filenames = glob.glob('Train_Images/*.jpg')
knn_clf = train(train_image_filenames)
#unknown_test_image_filenames = glob.glob('Test_Images_Unknown/*.jpg')

for idx, img in enumerate(unknown_test_image_filenames):
    X_img_unknown = face_recognition.load_image_file(img)
    X_enc_unknown = face_recognition.face_encodings(X_img)
    if len(X_enc_unknown) == 1:
        predictions_unknown = predict(X_img,knn_clf = knn_clf)
        output_unknown[idx] = predictions

print("The Google training set has a false positive rate of", len(output_unknown)/len(unknown_test_image_filenames))

The video training set has a false positive rate of 1.0
The Google training set has a false positive rate of 1.0


### Updating video with classification outputs

In [52]:
def knnToVideo(filename):
    
    length, input_movie, output_movie = readVideo(filename, "knn")
    known_faces, labels = readImageFiles("Train_ImagefromVideo")
    known_faces.resize(known_faces.shape[0], known_faces.shape[2])
    
    # Initialize some variables
    face_locations = []
    face_encodings = []
    face_names = []
    frame_number = 0
    knn_clf = train(train_image_filenames)
    
    #while input_movie.isOpened():
    while True: 
        # Grab a single frame of video
        ret, frame = input_movie.read()
        #print(frame.shape)
        frame_number += 1
        
        # Quit when the input video file ends
        if not ret:
            print("break")
            break

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_frame = frame[:, :, ::-1]

        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
        face_names = []
        for face_encoding in face_encodings:
            print("Found a face")
            # See if the face is a match for the known face(s)
            
            # could make this logic a lot prettier
            name = None
            predictions = predict(rgb_frame, knn_clf = knn_clf)
            if predictions[0][0] != 'unknown':
                name = predictions[0][0].split('/')[1].split('_')[0]

            face_names.append(name)

        # Label the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            if not name:
                continue

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

        # Write the resulting image to the output video file
        print("Writing frame {} / {}".format(frame_number, length))

        # confirmed: frames are being read fine, the issue is writing each frame
        output_movie.write(frame)

    print("All done")
    # All done!
    input_movie.release()
    output_movie.release()
    cv2.destroyAllWindows()
    

In [60]:
directToVideo("Test_Videos/MGAFE0008218--AS.01.mp4")
knnToVideo("Test_Videos/MGAFE0008218--AS.01.mp4")


Output filename: direct_MGAFE0008218--AS.01.mp4_output.avi
Writing frame 1 / 1603
Writing frame 2 / 1603
Writing frame 3 / 1603
Writing frame 4 / 1603
Writing frame 5 / 1603
Writing frame 6 / 1603
Writing frame 7 / 1603
Writing frame 8 / 1603
Writing frame 9 / 1603
Writing frame 10 / 1603
Writing frame 11 / 1603
Writing frame 12 / 1603
Writing frame 13 / 1603
Writing frame 14 / 1603
Writing frame 15 / 1603
Writing frame 16 / 1603
Writing frame 17 / 1603
Writing frame 18 / 1603
Writing frame 19 / 1603
Writing frame 20 / 1603
Writing frame 21 / 1603
Writing frame 22 / 1603
Writing frame 23 / 1603
Writing frame 24 / 1603
Writing frame 25 / 1603
Writing frame 26 / 1603
Writing frame 27 / 1603
Writing frame 28 / 1603
Writing frame 29 / 1603
Writing frame 30 / 1603
Writing frame 31 / 1603
Writing frame 32 / 1603
Writing frame 33 / 1603
Writing frame 34 / 1603
Writing frame 35 / 1603
Writing frame 36 / 1603
Writing frame 37 / 1603
Writing frame 38 / 1603
Writing frame 39 / 1603
Writing frame 

Found a face
Writing frame 259 / 1603
Found a face
Writing frame 260 / 1603
Found a face
Writing frame 261 / 1603
Writing frame 262 / 1603
Writing frame 263 / 1603
Writing frame 264 / 1603
Writing frame 265 / 1603
Writing frame 266 / 1603
Writing frame 267 / 1603
Writing frame 268 / 1603
Found a face
Writing frame 269 / 1603
Found a face
Writing frame 270 / 1603
Found a face
Writing frame 271 / 1603
Found a face
Writing frame 272 / 1603
Found a face
Writing frame 273 / 1603
Found a face
Writing frame 274 / 1603
Found a face
Writing frame 275 / 1603
Found a face
Writing frame 276 / 1603
Found a face
Writing frame 277 / 1603
Found a face
Writing frame 278 / 1603
Writing frame 279 / 1603
Writing frame 280 / 1603
Writing frame 281 / 1603
Found a face
Writing frame 282 / 1603
Found a face
Writing frame 283 / 1603
Found a face
Found a face
Writing frame 284 / 1603
Found a face
Found a face
Writing frame 285 / 1603
Found a face
Writing frame 286 / 1603
Writing frame 287 / 1603
Writing frame 2

Writing frame 480 / 1603
Writing frame 481 / 1603
Writing frame 482 / 1603
Writing frame 483 / 1603
Writing frame 484 / 1603
Writing frame 485 / 1603
Writing frame 486 / 1603
Writing frame 487 / 1603
Writing frame 488 / 1603
Writing frame 489 / 1603
Writing frame 490 / 1603
Writing frame 491 / 1603
Writing frame 492 / 1603
Writing frame 493 / 1603
Writing frame 494 / 1603
Writing frame 495 / 1603
Writing frame 496 / 1603
Writing frame 497 / 1603
Writing frame 498 / 1603
Writing frame 499 / 1603
Writing frame 500 / 1603
Writing frame 501 / 1603
Writing frame 502 / 1603
Writing frame 503 / 1603
Writing frame 504 / 1603
Writing frame 505 / 1603
Writing frame 506 / 1603
Writing frame 507 / 1603
Writing frame 508 / 1603
Writing frame 509 / 1603
Writing frame 510 / 1603
Writing frame 511 / 1603
Writing frame 512 / 1603
Writing frame 513 / 1603
Writing frame 514 / 1603
Writing frame 515 / 1603
Writing frame 516 / 1603
Writing frame 517 / 1603
Writing frame 518 / 1603
Writing frame 519 / 1603


Found a face
Found a face
Writing frame 804 / 1603
Found a face
Found a face
Writing frame 805 / 1603
Found a face
Found a face
Writing frame 806 / 1603
Found a face
Found a face
Writing frame 807 / 1603
Found a face
Found a face
Writing frame 808 / 1603
Found a face
Found a face
Writing frame 809 / 1603
Found a face
Found a face
Writing frame 810 / 1603
Found a face
Found a face
Writing frame 811 / 1603
Found a face
Found a face
Writing frame 812 / 1603
Found a face
Writing frame 813 / 1603
Found a face
Writing frame 814 / 1603
Found a face
Writing frame 815 / 1603
Found a face
Writing frame 816 / 1603
Found a face
Writing frame 817 / 1603
Found a face
Writing frame 818 / 1603
Found a face
Writing frame 819 / 1603
Found a face
Writing frame 820 / 1603
Found a face
Writing frame 821 / 1603
Found a face
Writing frame 822 / 1603
Found a face
Writing frame 823 / 1603
Found a face
Writing frame 824 / 1603
Found a face
Writing frame 825 / 1603
Found a face
Writing frame 826 / 1603
Found a f

Found a face
Writing frame 1077 / 1603
Found a face
Writing frame 1078 / 1603
Found a face
Writing frame 1079 / 1603
Found a face
Writing frame 1080 / 1603
Writing frame 1081 / 1603
Writing frame 1082 / 1603
Writing frame 1083 / 1603
Writing frame 1084 / 1603
Writing frame 1085 / 1603
Writing frame 1086 / 1603
Writing frame 1087 / 1603
Writing frame 1088 / 1603
Writing frame 1089 / 1603
Writing frame 1090 / 1603
Writing frame 1091 / 1603
Writing frame 1092 / 1603
Writing frame 1093 / 1603
Writing frame 1094 / 1603
Writing frame 1095 / 1603
Writing frame 1096 / 1603
Writing frame 1097 / 1603
Writing frame 1098 / 1603
Writing frame 1099 / 1603
Writing frame 1100 / 1603
Writing frame 1101 / 1603
Writing frame 1102 / 1603
Writing frame 1103 / 1603
Writing frame 1104 / 1603
Writing frame 1105 / 1603
Writing frame 1106 / 1603
Writing frame 1107 / 1603
Writing frame 1108 / 1603
Writing frame 1109 / 1603
Writing frame 1110 / 1603
Writing frame 1111 / 1603
Writing frame 1112 / 1603
Writing fram

...

Writing frame 429 / 1603
Writing frame 430 / 1603
Writing frame 431 / 1603
Writing frame 432 / 1603
Writing frame 433 / 1603
Writing frame 434 / 1603
Writing frame 435 / 1603
Writing frame 436 / 1603
Writing frame 437 / 1603
Writing frame 438 / 1603
Writing frame 439 / 1603
Writing frame 440 / 1603
Writing frame 441 / 1603
Writing frame 442 / 1603
Writing frame 443 / 1603
Writing frame 444 / 1603
Writing frame 445 / 1603
Writing frame 446 / 1603
Writing frame 447 / 1603
Writing frame 448 / 1603
Writing frame 449 / 1603
Writing frame 450 / 1603
Writing frame 451 / 1603
Writing frame 452 / 1603
Writing frame 453 / 1603
Writing frame 454 / 1603
Writing frame 455 / 1603
Writing frame 456 / 1603
Writing frame 457 / 1603
Writing frame 458 / 1603
Writing frame 459 / 1603
Writing frame 460 / 1603
Writing frame 461 / 1603
Writing frame 462 / 1603
Writing frame 463 / 1603
Writing frame 464 / 1603
Writing frame 465 / 1603
Writing frame 466 / 1603
Writing frame 467 / 1603
Writing frame 468 / 1603


Writing frame 757 / 1603
Writing frame 758 / 1603
Writing frame 759 / 1603
Writing frame 760 / 1603
Writing frame 761 / 1603
Writing frame 762 / 1603
Writing frame 763 / 1603
Writing frame 764 / 1603
Writing frame 765 / 1603
Writing frame 766 / 1603
Writing frame 767 / 1603
Writing frame 768 / 1603
Writing frame 769 / 1603
Writing frame 770 / 1603
Writing frame 771 / 1603
Writing frame 772 / 1603
Writing frame 773 / 1603
Writing frame 774 / 1603
Writing frame 775 / 1603
Writing frame 776 / 1603
Writing frame 777 / 1603
Writing frame 778 / 1603
Writing frame 779 / 1603
Writing frame 780 / 1603
Writing frame 781 / 1603
Writing frame 782 / 1603
Writing frame 783 / 1603
Writing frame 784 / 1603
Writing frame 785 / 1603
Writing frame 786 / 1603
Writing frame 787 / 1603
Writing frame 788 / 1603
Writing frame 789 / 1603
Writing frame 790 / 1603
Writing frame 791 / 1603
Writing frame 792 / 1603
Found a face
Writing frame 793 / 1603
Found a face
Writing frame 794 / 1603
Found a face
Writing fra

Writing frame 1039 / 1603
Writing frame 1040 / 1603
Writing frame 1041 / 1603
Found a face
Writing frame 1042 / 1603
Found a face
Writing frame 1043 / 1603
Found a face
Writing frame 1044 / 1603
Found a face
Writing frame 1045 / 1603
Found a face
Writing frame 1046 / 1603
Found a face
Writing frame 1047 / 1603
Found a face
Writing frame 1048 / 1603
Writing frame 1049 / 1603
Writing frame 1050 / 1603
Writing frame 1051 / 1603
Writing frame 1052 / 1603
Writing frame 1053 / 1603
Writing frame 1054 / 1603
Found a face
Writing frame 1055 / 1603
Found a face
Writing frame 1056 / 1603
Found a face
Writing frame 1057 / 1603
Found a face
Writing frame 1058 / 1603
Found a face
Writing frame 1059 / 1603
Writing frame 1060 / 1603
Writing frame 1061 / 1603
Found a face
Writing frame 1062 / 1603
Found a face
Writing frame 1063 / 1603
Found a face
Writing frame 1064 / 1603
Found a face
Writing frame 1065 / 1603
Found a face
Writing frame 1066 / 1603
Found a face
Writing frame 1067 / 1603
Found a face

Writing frame 1309 / 1603
Writing frame 1310 / 1603
Writing frame 1311 / 1603
Writing frame 1312 / 1603
Writing frame 1313 / 1603
Writing frame 1314 / 1603
Writing frame 1315 / 1603
Writing frame 1316 / 1603
Writing frame 1317 / 1603
Writing frame 1318 / 1603
Writing frame 1319 / 1603
Writing frame 1320 / 1603
Writing frame 1321 / 1603
Writing frame 1322 / 1603
Writing frame 1323 / 1603
Writing frame 1324 / 1603
Writing frame 1325 / 1603
Writing frame 1326 / 1603
Writing frame 1327 / 1603
Writing frame 1328 / 1603
Writing frame 1329 / 1603
Writing frame 1330 / 1603
Writing frame 1331 / 1603
Writing frame 1332 / 1603
Writing frame 1333 / 1603
Writing frame 1334 / 1603
Writing frame 1335 / 1603
Writing frame 1336 / 1603
Writing frame 1337 / 1603
Writing frame 1338 / 1603
Writing frame 1339 / 1603
Writing frame 1340 / 1603
Writing frame 1341 / 1603
Writing frame 1342 / 1603
Writing frame 1343 / 1603
Writing frame 1344 / 1603
Writing frame 1345 / 1603
Writing frame 1346 / 1603
Writing fram

Found a face
Found a face
Writing frame 1526 / 1603
Found a face
Found a face
Writing frame 1527 / 1603
Found a face
Found a face
Writing frame 1528 / 1603
Found a face
Found a face
Writing frame 1529 / 1603
Found a face
Found a face
Writing frame 1530 / 1603
Found a face
Found a face
Writing frame 1531 / 1603
Found a face
Found a face
Writing frame 1532 / 1603
Found a face
Found a face
Writing frame 1533 / 1603
Found a face
Found a face
Writing frame 1534 / 1603
Found a face
Found a face
Writing frame 1535 / 1603
Found a face
Found a face
Writing frame 1536 / 1603
Found a face
Found a face
Writing frame 1537 / 1603
Found a face
Found a face
Writing frame 1538 / 1603
Found a face
Found a face
Writing frame 1539 / 1603
Found a face
Found a face
Writing frame 1540 / 1603
Found a face
Found a face
Writing frame 1541 / 1603
Found a face
Found a face
Writing frame 1542 / 1603
Found a face
Found a face
Writing frame 1543 / 1603
Found a face
Writing frame 1544 / 1603
Found a face
Writing fram